<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/wikidata_sparql_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sparqlwrapper

# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 348kB 11.6MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


In [0]:
import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import re
from collections import Counter

def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [80]:
endpoint_url = "https://query.wikidata.org/sparql"

query_noodle = """SELECT ?noodle ?noodleLabel ?country_of_origin ?country_of_originLabel ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel  ?Commons_category 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "   [AUTO_LANGUAGE],en". }
  ?noodle wdt:P279 wd:Q192874.
  OPTIONAL { ?noodle wdt:P495 ?country_of_origin. }
  OPTIONAL { ?noodle wdt:P31 ?instance_of. }
  OPTIONAL { ?noodle wdt:P279 ?subclass_of. }
  OPTIONAL { ?noodle wdt:P373 ?Commons_category. }  
}"""


df_noodle = get_results(endpoint_url, query_noodle)
df_noodle.head()

,noodle,noodleLabel,country_of_origin,country_of_originLabel,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,Commons_category
0,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
1,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
2,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
3,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q1142,Alsace,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
4,http://www.wikidata.org/entity/Q34156,Korean noodles,http://www.wikidata.org/entity/Q18097,Korea,None,None,http://www.wikidata.org/entity/Q192874,noodle,Noodles from Korea


In [99]:
noodle_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_noodle.groupby(\
                                                     ['noodleLabel'])['country_of_originLabel']\
                    .agg('count').index.values.tolist()])

noodle_list = sorted([re.sub("\-", " ",item) for item in noodle_list])
noodle_counter = Counter(noodle_list)

noodle_list = []
for key in noodle_counter.keys():
  if len(key) != 0:
    noodle_list.append(key)
print(noodle_list)

['bakmi', 'black noodles', 'boat noodles', 'buckwheat noodles', 'buldak bokkeum myun', 'bánh hỏi', 'cart noodle', 'cellophane noodles', 'char kway teow', 'chinese noodles', 'chinkiang pot cover noodles', 'cold noodle', 'curry noodle', 'dragon beard noodles', 'extruded noodle', 'fried noodles', 'hokkien mee', 'hot dry noodles', 'jajangmyeon', 'japanese noodles', 'kadaif noodles', 'kesme', 'kishimen', 'knife cut noodle', 'korean noodles', 'laghman', 'mi rebus', 'mie ayam', 'mì', 'narrow lapsha', 'pancit', 'phat si io', 'ramen', 'ribbon noodle', 'rice noodles', 'singapore chow mein', 'soba', 'spätzle', 'sōmen', 'udon', 'vietnamese noodles', 'wide lapsha', 'wonton noodles', 'zhajiangmian']


In [6]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel']]['noodleLabel'].unique()

array(['Spätzle', 'Korean noodles', 'zhajiangmian', 'ramen',
       'Curry noodle', 'udon', 'sōmen', 'soba', 'hot dry noodles',
       'cart noodle', 'cellophane noodles', 'jajangmyeon', 'Bakmi',
       'rice noodles', 'Hokkien mee', 'fried noodles', 'Pancit',
       'Chinese noodles', 'Wonton noodles', 'buckwheat noodles', 'kesme',
       'black noodles', 'Bánh hỏi', 'Char kway teow',
       'Chinkiang pot cover noodles', 'Mi rebus', 'dragon beard noodles',
       'phat si io', 'Vietnamese noodles', 'kishimen', 'Kadaif noodles',
       'cold noodle', 'Buldak Bokkeum Myun', 'Mie ayam',
       'Japanese noodles', 'boat noodles', 'Q24933621',
       'Singapore chow mein', 'Q27566431', 'narrow lapsha', 'wide lapsha',
       'laghman', 'extruded noodle', 'knife-cut noodle', 'Mì',
       'ribbon noodle'], dtype=object)

In [7]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel','Commons_category']]['Commons_category'].unique()

array(['Spaetzle', 'Noodles from Korea', 'Zha jiang mian', 'Ramen', None,
       'Udon', 'Somen', 'Soba', 'Re gan mian', 'Cart noodle',
       'Cellophane noodles', 'Jajangmyeon', 'Noodles from Indonesia',
       'Rice noodle', 'Hokkien mee', 'Fried noodles', 'Pancit',
       'Noodles from China', 'Wonton noodle', 'Kesme', 'Bánh hỏi',
       'Char kway teow', 'Bakmi Jawa', 'Phat si-io',
       'Noodles from Vietnam', 'Kishimen', 'Tel kadayıf', 'Cold noodles',
       'Mie ayam', 'Noodles from Japan', 'Laghman', 'Extruded noodles',
       'Knife-cut noodles'], dtype=object)

In [0]:
df.groupby(['Commons_category','noodleLabel'])['noodleLabel'].agg('count')

Commons_category        noodleLabel       
Bakmi Jawa              Mi rebus              1
Bánh hỏi                Bánh hỏi              1
Cart noodle             cart noodle           2
Cellophane noodles      cellophane noodles    2
Char kway teow          Char kway teow        1
Cold noodles            cold noodle           1
Extruded noodles        extruded noodle       1
Fried noodles           fried noodles         2
Hokkien mee             Hokkien mee           2
Jajangmyeon             jajangmyeon           3
Kesme                   kesme                 2
Kishimen                kishimen              2
Knife-cut noodles       knife-cut noodle      1
Laghman                 laghman               2
Mie ayam                Mie ayam              3
Noodles from China      Chinese noodles       1
Noodles from Indonesia  Bakmi                 2
Noodles from Japan      Japanese noodles      1
Noodles from Korea      Korean noodles        1
Noodles from Vietnam    Vietnamese noodles   

In [71]:
endpoint_url = "https://query.wikidata.org/sparql"
query_soup = """SELECT ?subclass ?subclassLabel ?TasteAtlas_ID ?instance_of ?instance_ofLabel ?country_of_origin ?country_of_originLabel ?topic_s_main_template ?topic_s_main_templateLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?subclass wdt:P279 wd:Q41415.
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  OPTIONAL { ?subclass wdt:P495 ?country_of_origin. }
  OPTIONAL { ?subclass wdt:P1424 ?topic_s_main_template. }
}"""

df_soup = get_results(endpoint_url, query_soup)
df_soup.head()

,subclass,subclassLabel,TasteAtlas_ID,instance_of,instance_ofLabel,country_of_origin,country_of_originLabel,topic_s_main_template,topic_s_main_templateLabel
0,http://www.wikidata.org/entity/Q11179944,Stewed dog meat with soft-shelled turtle,None,None,None,http://www.wikidata.org/entity/Q148,People's Republic of China,None,None
1,http://www.wikidata.org/entity/Q11242258,Soto mie,None,http://www.wikidata.org/entity/Q2095,food,http://www.wikidata.org/entity/Q252,Indonesia,None,None
2,http://www.wikidata.org/entity/Q11265947,Kenoshiru,None,None,None,http://www.wikidata.org/entity/Q17,Japan,None,None
3,http://www.wikidata.org/entity/Q11266028,Kenchinjiru,None,None,None,None,None,None,None
4,http://www.wikidata.org/entity/Q11443995,taipien,None,None,None,None,None,None,None


In [76]:
soup_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_soup.groupby(\
                                                     ['subclassLabel'])['country_of_originLabel']\
                    .agg('count').index.values.tolist()])

soup_list = sorted([re.sub("\-", " ",item) for item in soup_list])
soup_counter = Counter(soup_list)

soup_list = []
for key in soup_counter.keys():
  if len(key) != 0:
    soup_list.append(key)
print(soup_list)

['15 bean soup', '2007 vietnam food scare', 'acquacotta', 'aguadito de pollo', 'ajoblanco', 'alicot', 'amiedi', 'amish preaching soup', 'arabaşı soup', 'ash reshteh', 'ashe doogh', 'asian soup', 'aush', 'bacon soup', 'baeksuk', 'bagnun', 'banga', 'batchoy', 'bean soup', 'beer soup', 'beetroot soup', 'beyran', 'binignit', "bird's nest soup", 'bisque', 'black sesame soup', 'black soup', 'blood soup', 'bob chorba', 'bogeo', 'bookbinder soup', 'borş de burechiuşe', 'borș', 'bosintang', 'bouillon', 'bouillon cube', 'bouneschlupp', 'brown windsor soup', 'budae jjigae', 'buddha jumps over the wall', 'bun rieu', 'buridda', 'bún bò huế', 'bún mắm', 'bún ốc', 'cabbage soup', 'caldillo de perro', 'caldo de costilla', 'caldo de pollo', 'caldo de siete mares', 'caldo gallego', 'caldo tlalpeño', 'caldo verde', 'canja de goa', 'cantonese seafood soup', 'caparrones', 'cazuela', 'celimpungan', 'chakna', 'changua', 'chankonabe', 'cheese soup', 'chicken and dumplings', 'chicken soup', 'chikhirtma', 'chin

In [77]:
len(soup_list)

363

In [65]:
endpoint_url = "https://query.wikidata.org/sparql"
query_rice_dish = """SELECT ?is_a_list_of ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel ?country_of_origin ?country_of_originLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?is_a_list_of wdt:P279 wd:Q21976260.
  OPTIONAL {  }
  OPTIONAL { ?is_a_list_of wdt:P5456 ?instance_of. }
  OPTIONAL { ?is_a_list_of wdt:P279 ?subclass_of. }
  OPTIONAL { ?is_a_list_of wdt:P495 ?country_of_origin. }
}
"""

df_rice_dish = get_results(endpoint_url,query_rice_dish)
df_rice_dish.head()

,is_a_list_of,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,country_of_origin,country_of_originLabel
0,http://www.wikidata.org/entity/Q19029,arroz-con-leche,arroz-con-leche,http://www.wikidata.org/entity/Q182940,dessert,None,None
1,http://www.wikidata.org/entity/Q19029,arroz-doce,arroz-doce,http://www.wikidata.org/entity/Q182940,dessert,None,None
2,http://www.wikidata.org/entity/Q19029,sutlijas,sutlijas,http://www.wikidata.org/entity/Q182940,dessert,None,None
3,http://www.wikidata.org/entity/Q19029,arroz-con-leche,arroz-con-leche,http://www.wikidata.org/entity/Q186817,porridge,None,None
4,http://www.wikidata.org/entity/Q19029,arroz-doce,arroz-doce,http://www.wikidata.org/entity/Q186817,porridge,None,None


In [66]:
df_rice_dish['instance_ofLabel']

0      arroz-con-leche
1           arroz-doce
2             sutlijas
3      arroz-con-leche
4           arroz-doce
            ...       
207               None
208              sarma
209               None
210               None
211               None
Name: instance_ofLabel, Length: 212, dtype: object

In [69]:
df_rice_dish.groupby(['subclass_ofLabel', 'instance_ofLabel'], as_index=False)['country_of_originLabel'].agg('count').instance_ofLabel.values.tolist()

rice_dish_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                         for item in df_rice_dish.groupby(\
                                                          ['subclass_ofLabel', 'instance_ofLabel'], as_index=False)\
                         ['country_of_originLabel'].agg('count').instance_ofLabel.values.tolist()])

rice_dish_list = sorted([re.sub("\-", " ",item) for item in rice_dish_list])
rice_dish_counter = Counter(rice_dish_list)

rice_dish_list = []
for key in rice_dish_counter.keys():
  if len(key) != 0:
    rice_dish_list.append(key)
print(rice_dish_list)

['akki rotti', 'arroz caldo', 'arroz carreteiro', 'arroz con coco', 'arroz con leche', 'arroz con pollo', 'arroz doce', 'bibimbap', 'biryani', 'botamochi', 'cabidela', 'calas', 'chazuke', 'chukadon', 'com lam', 'daifuku', 'dal bhat', 'donburi', 'gyudon', 'hayashi rice', 'hoedeopbap', 'hokkien fried rice', 'idli', 'kabsa', 'kamameshi', 'kayu', 'ketupat', 'kheer', 'kiribath', 'lemang', 'loco moco', 'lotus leaf rice', 'mansaf', 'midye dolma', 'mujaddara', 'nasi campur', 'nasi kerabu', 'nasi lemak', 'oyakodon', 'pabellon criollo', 'paella', 'panta bhat', 'perde pilavi', 'platillo moros y cristiano', 'pongal', 'red beans and rice', 'risotto', 'sarma', 'sekihan', 'sindhi biryani', 'sushi', 'sutlijas', 'tacu tacu', 'tamago kake gohan', 'tekkadon', 'unadon', 'warabimochi', 'xoi', 'yangzhou fried rice', 'zongzi', 'zosui']


In [37]:
endpoint_url = "https://query.wikidata.org/sparql"
query_salad = """
SELECT ?subclass ?subclassLabel ?TasteAtlas_ID ?instance_of ?instance_ofLabel ?country_of_origin ?country_of_originLabel ?topic_s_main_template ?topic_s_main_templateLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?subclass wdt:P279 wd:Q9266.
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  OPTIONAL { ?subclass wdt:P495 ?country_of_origin. }
  OPTIONAL { ?subclass wdt:P1424 ?topic_s_main_template. }
}
"""

df_salad = get_results(endpoint_url,query_salad)
df_salad.head()



,subclass,subclassLabel,TasteAtlas_ID,instance_of,instance_ofLabel,country_of_origin,country_of_originLabel,topic_s_main_template,topic_s_main_templateLabel
0,http://www.wikidata.org/entity/Q6030071,Gavurdağı salatası,None,http://www.wikidata.org/entity/Q2095,food,http://www.wikidata.org/entity/Q43,Turkey,None,None
1,http://www.wikidata.org/entity/Q6060108,Q6060108,None,None,None,None,None,None,None
2,http://www.wikidata.org/entity/Q6076597,Q6076597,pipirrana,None,None,None,None,None,None
3,http://www.wikidata.org/entity/Q6117451,Q6117451,None,None,None,None,None,None,None
4,http://www.wikidata.org/entity/Q6128321,Salad cream,None,None,None,http://www.wikidata.org/entity/Q145,United Kingdom,None,None


In [64]:
salad_list = sorted([re.sub("q[0-9]+","",item.lower()) for item in df_salad.subclassLabel.values.tolist()])
salad_counter = Counter(salad_list)

salad_list = []
for key in salad_counter.keys():
  if len(key) != 0:
    salad_list.append(key)
print(salad_list)

['a thoke', 'acar', 'ahtapot salata', 'arab salad', 'asinan', 'baba ghanoush', 'bean salad', 'beetroot salad', 'beyin salata', 'broccoli slaw', 'buljol', 'caesar salad', 'caprese salad', 'carrot salad', 'celery root salad', 'celery victor', 'cheese slaw', 'chef salad', 'chicken salad', 'chilean salad', 'chinese chicken salad', 'cobb salad', 'coleslaw', 'composed salad', 'cookie salad', 'crab louie', 'curtido', 'dakos', 'dessert salad', 'dressed herring', 'egg salad', 'eggplant salads and appetizers', 'egg–anchovy salad', 'eruca vesicaria', 'escalivada', 'esgarrat', 'esqueixada', 'fattoush', 'fiambre', 'fruit salad', 'gado-gado', 'garden salad', 'gavurdağı salatası', 'glass noodle salad', 'glasswort salad', 'glorified rice', 'golbaengi-muchim', 'goma-ae', 'greek salad', 'ham salad', 'herring salad', 'hmiss', 'israeli eggplant salad', 'israeli salad', 'jello salad', 'karedok', 'kelan antep', 'kinilnat', 'korean carrot salad', 'kosambari', 'kuluban', 'kurkkusalaatti', 'kısır', 'lalab', 'l

In [70]:
rice_dish_list

['akki rotti',
 'arroz caldo',
 'arroz carreteiro',
 'arroz con coco',
 'arroz con leche',
 'arroz con pollo',
 'arroz doce',
 'bibimbap',
 'biryani',
 'botamochi',
 'cabidela',
 'calas',
 'chazuke',
 'chukadon',
 'com lam',
 'daifuku',
 'dal bhat',
 'donburi',
 'gyudon',
 'hayashi rice',
 'hoedeopbap',
 'hokkien fried rice',
 'idli',
 'kabsa',
 'kamameshi',
 'kayu',
 'ketupat',
 'kheer',
 'kiribath',
 'lemang',
 'loco moco',
 'lotus leaf rice',
 'mansaf',
 'midye dolma',
 'mujaddara',
 'nasi campur',
 'nasi kerabu',
 'nasi lemak',
 'oyakodon',
 'pabellon criollo',
 'paella',
 'panta bhat',
 'perde pilavi',
 'platillo moros y cristiano',
 'pongal',
 'red beans and rice',
 'risotto',
 'sarma',
 'sekihan',
 'sindhi biryani',
 'sushi',
 'sutlijas',
 'tacu tacu',
 'tamago kake gohan',
 'tekkadon',
 'unadon',
 'warabimochi',
 'xoi',
 'yangzhou fried rice',
 'zongzi',
 'zosui']

In [100]:
endpoint_url = "https://query.wikidata.org/sparql"
query_dish = """
SELECT ?subclass ?TasteAtlas_ID ?subclass_of ?subclass_ofLabel ?instance_of ?instance_ofLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
  OPTIONAL { ?subclass wdt:P5456 ?TasteAtlas_ID. }
  OPTIONAL { ?subclass wdt:P279 ?subclass_of. }
  OPTIONAL { ?subclass wdt:P31 ?instance_of. }
  
  ?subclass wdt:P279 wd:Q746549.
}"""
df_dish = get_results(endpoint_url,query_dish)
df_dish.head()

,subclass,TasteAtlas_ID,subclass_of,subclass_ofLabel,instance_of,instance_ofLabel
0,http://www.wikidata.org/entity/Q177,pizza,http://www.wikidata.org/entity/Q666242,flatbread,http://www.wikidata.org/entity/Q19861951,type of food or dish
1,http://www.wikidata.org/entity/Q177,pizza,http://www.wikidata.org/entity/Q746549,dish,http://www.wikidata.org/entity/Q19861951,type of food or dish
2,http://www.wikidata.org/entity/Q177,pizza,http://www.wikidata.org/entity/Q3245975,finished good,http://www.wikidata.org/entity/Q19861951,type of food or dish
3,http://www.wikidata.org/entity/Q177,pizza,http://www.wikidata.org/entity/Q13485782,convenience food,http://www.wikidata.org/entity/Q19861951,type of food or dish
4,http://www.wikidata.org/entity/Q177,pizza,http://www.wikidata.org/entity/Q26996677,"Pizzas, casseroles",http://www.wikidata.org/entity/Q19861951,type of food or dish


In [104]:
df_dish.groupby(['TasteAtlas_ID', 'subclass_ofLabel'], as_index=False)['subclass'].agg('count').TasteAtlas_ID.values.tolist()

['a-gei',
 'agedashi-tofu',
 'ajapsandali',
 'alheira',
 'aloo-gobi',
 'aloo-gobi',
 'aloo-gobi',
 'amatriciana',
 'amatriciana',
 'ambuyat',
 'ambuyat',
 'anticucho',
 'aperitivo',
 'apohtin',
 'apohtin',
 'apohtin',
 'apohtin',
 'apohtin',
 'apple-sauce',
 'apple-sauce',
 'arroz-con-coco',
 'arroz-con-coco',
 'arroz-junto',
 'arroz-rojo',
 'arroz-rojo',
 'aushak',
 'avocado-toast',
 'avocado-toast',
 'bacalhau-a-bras',
 'bacalhau-a-bras',
 'bacon-egg-and-cheese -sandwich',
 'bacon-egg-and-cheese -sandwich',
 'badrijani',
 'baingan-bharta',
 'baked-beans',
 'balut',
 'balut',
 'banana-bread',
 'banana-bread',
 'banku',
 'bannock',
 'bannock',
 'banosh',
 'barreado',
 'beef-bourguignon',
 'beef-chow-fun',
 'beef-chow-fun',
 'beef-noodle-soup',
 'beef-noodle-soup',
 'beef-noodle-soup',
 'beef-stroganoff',
 'beef-stroganoff',
 'beef-wellington',
 'beef-wellington',
 'beignets',
 'beignets',
 'bethmannchen',
 'bethmannchen',
 'biryani',
 'biryani',
 'biryani',
 'biryani',
 'biryani',
 'bi

In [106]:
dish_list = sorted([re.sub("q[0-9]+","",item.lower()) \
                    for item in df_dish.groupby(\
                                                ['TasteAtlas_ID', 'subclass_ofLabel'], as_index=False)\
                    .agg('count').TasteAtlas_ID.values.tolist()])

dish_list = sorted([re.sub("\-", " ",item) for item in dish_list])

dish_counter = Counter(dish_list)

dish_list = []
for key in dish_counter.keys():
  if len(key) != 0:
    dish_list.append(key)
print(dish_list)

['a gei', 'agedashi tofu', 'ajapsandali', 'alheira', 'aloo gobi', 'amatriciana', 'ambuyat', 'anticucho', 'aperitivo', 'apohtin', 'apple sauce', 'arroz con coco', 'arroz junto', 'arroz rojo', 'aushak', 'avocado toast', 'bacalhau a bras', 'bacon egg and cheese  sandwich', 'badrijani', 'baingan bharta', 'baked beans', 'balut', 'banana bread', 'banku', 'bannock', 'banosh', 'barreado', 'beef bourguignon', 'beef chow fun', 'beef noodle soup', 'beef stroganoff', 'beef wellington', 'beignets', 'bethmannchen', 'biryani', 'bison burger', 'black peas', 'black sesame soup', 'bolo do caco', 'bouillabaisse', 'briam', 'bruschetta', 'bubur ayam', 'bun cha', 'burger', 'butadon', 'cacik', 'calulu', 'cantonese seafood soup', 'capuns', 'carapulcra', 'carcerato', 'casunziei', 'cauliflower cheese', 'century egg', 'cevapi', 'cha siu bao', 'chahan', 'chairo', 'chanpuru', 'chen mapo doufu', 'chistorra', 'choucroute garnie', 'chow mein', 'chukadon', 'ciccioli', 'ciorba de burta', 'cock a leekie', 'confit de can

In [120]:
full_list = sorted(dish_list + rice_dish_list + soup_list + noodle_list + salad_list)
full_list = sorted([item for item in full_list if not item[0].isdigit()])

full_counter = Counter(full_list)
full_list=[]
for key in full_counter.keys():
  if len(key) != 0:
    full_list.append(key)
print(len(full_list))

871


In [0]:
dish_keywords = np.array(full_list)
np.savez("dish_keywords", dish_keywords)

In [0]:
"label": "DISH", "pattern": [{'LOWER': 'agedashi'}, {'LOWER': 'tofu'}]

In [0]:
major_list = []
for item in full_list[:10]:
  pattern = []
  line = {}
  for word in item.split():
    pattern.append('{"LOWER": "' + word + '"}')
#  line = '{"label": "DISH", "pattern": ' + str(pattern) + '}'
  line['label'] = "DISH"
  line['pattern'] = pattern

  major_list.append(line)

#  major_list.append(re.sub("\\'","",line))

In [148]:
major_list

[{'label': 'DISH', 'pattern': ['{"LOWER": "a"}', '{"LOWER": "gei"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "a"}', '{"LOWER": "thoke"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "acar"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "acquacotta"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "agedashi"}', '{"LOWER": "tofu"}']},
 {'label': 'DISH',
  'pattern': ['{"LOWER": "aguadito"}',
   '{"LOWER": "de"}',
   '{"LOWER": "pollo"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "ahtapot"}', '{"LOWER": "salata"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "ajapsandali"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "ajoblanco"}']},
 {'label': 'DISH', 'pattern': ['{"LOWER": "akki"}', '{"LOWER": "rotti"}']}]

In [124]:
full_list[:30]

['a gei',
 'a thoke',
 'acar',
 'acquacotta',
 'agedashi tofu',
 'aguadito de pollo',
 'ahtapot salata',
 'ajapsandali',
 'ajoblanco',
 'akki rotti',
 'alheira',
 'alicot',
 'aloo gobi',
 'amatriciana',
 'ambuyat',
 'amiedi',
 'amish preaching soup',
 'anticucho',
 'aperitivo',
 'apohtin',
 'apple sauce',
 'arab salad',
 'arabaşı soup',
 'arroz caldo',
 'arroz carreteiro',
 'arroz con coco',
 'arroz con leche',
 'arroz con pollo',
 'arroz doce',
 'arroz junto']